In [170]:
import sys
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd

# First Level Crawling

In [165]:
def scrape_links(url):
    url_list=[]

    # import the webdriver
    driver = webdriver.Chrome()
    driver.get(url)

    # Get scroll height
    scroll = driver.find_element(By.XPATH,"/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]")
    last_height = driver.execute_script("return arguments[0].scrollHeight",scroll)

    while True:
        elems = driver.find_elements(By.XPATH, "//a[@class='hfpxzc']")
        for elem in elems:
            name = elem.get_attribute("aria-label")
            link = elem.get_attribute("href")
            url_list.append({"Name":name,"Link":link})
        driver.execute_script("arguments[0].scrollTo(0,arguments[0].scrollHeight);",scroll)
        time.sleep(5)
        new_height = driver.execute_script("return arguments[0].scrollHeight",scroll)
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
#Get List of Restaurant & Links
url = "https://www.google.com/maps/search/restaurants+in+singapore/@1.294855,103.8136815,14z/data=!3m1!4b1?entry=ttu"
scrape_links(url)
links_to_restaurants = pd.DataFrame(url_list)
links_to_restaurants
unique_links_to_restaurants = links_to_restaurants.groupby(links_to_restaurants.columns.tolist(),as_index=False).size()
unique_links_to_restaurants.to_csv("links_to_restaurants.csv")

# Second Level Crawling

In [231]:
def scrape_reviews(url, placeName):
    # Create an empty list to store dictionaries
    data_list = []
    found_elements = set()

    # Import the webdriver
    driver = webdriver.Chrome()
    driver.get(url)

    reviewButton = driver.find_element(By.XPATH, "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]") 
    print(reviewButton.get_attribute('aria-label'))
    time.sleep(5)
    reviewButton.click()
    time.sleep(5)

    # Get scroll height
    scroll = driver.find_element(By.XPATH,"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")
    last_height = driver.execute_script("return arguments[0].scrollHeight",scroll)

    while True:
        elems = driver.find_elements(By.XPATH, "//div[@class='jftiEf fontBodyMedium ']")
        for elem in elems:
            name = elem.get_attribute("aria-label")
            if name in found_elements:
                continue
            found_elements.add(name)
            smaller_elems = elem.find_elements(By.XPATH, ".//span[@class='kvMYJc']")
            ratings = [x.get_attribute("aria-label") for x in smaller_elems]
            tests = elem.find_elements(By.XPATH, ".//div[@class='MyEned']")
            for i, test in enumerate(tests):
                reviews = test.find_elements(By.XPATH, ".//span[@class='wiI7pd']")
                for review in reviews:
                    review_text = review.get_attribute("innerHTML")
                    data_list.append({"Name": name, "Rating": ratings[i], "Review": review_text})
        driver.execute_script("arguments[0].scrollTo(0,arguments[0].scrollHeight);",scroll)
        time.sleep(5)
        new_height = driver.execute_script("return arguments[0].scrollHeight",scroll)
        if new_height == last_height:
            break
        last_height = new_height
    df = pd.DataFrame(data_list)
    # df = df.groupby(df.columns.tolist(),as_index=False).size()
    # df = df.iloc[:,0:3]
    df.to_csv(f"reviews_for_{placeName}.csv")

In [237]:
test = unique_links_to_restaurants.iloc[7:]
test = test.reset_index()
test = test.iloc[:,1:3]
test

,Name,Link
0,Bar-A-Thym Restaurant by Francois Mermilliod,https://www.google.com/maps/place/Bar-A-Thym+R...
1,Bar-Roque Grill,https://www.google.com/maps/place/Bar-Roque+Gr...
2,Baristart Coffee Singapore (Wisma Atria),https://www.google.com/maps/place/Baristart+Co...
3,Beng Thin Hoon Kee Restaurant,https://www.google.com/maps/place/Beng+Thin+Ho...
4,Birds of a Feather,https://www.google.com/maps/place/Birds+of+a+F...
...,...,...
110,Yan Restaurant at National Gallery Singapore 宴中餐馆,https://www.google.com/maps/place/Yan+Restaura...
111,Yardbird Southern Table and Bar,https://www.google.com/maps/place/Yardbird+Sou...
112,Yellow Pot Tanjong Pagar,https://www.google.com/maps/place/Yellow+Pot+T...
113,Zén,https://www.google.com/maps/place/Z%C3%A9n/dat...


In [238]:
for i in range(len(test)):
    name = test["Name"][i]
    url = test["Link"][i]
    scrape_reviews(url,name)

Reviews for Bar-A-Thym Restaurant by Francois Mermilliod
Reviews for Bar-Roque Grill
Reviews for Baristart Coffee Singapore (Wisma Atria)
Reviews for Beng Thin Hoon Kee Restaurant
Reviews for Birds of a Feather
Reviews for Blossom Restaurant
Reviews for Braci
Reviews for Bread Street Kitchen by Gordon Ramsay
Reviews for Buona Terra
Reviews for CENZO
Reviews for Candlenut
Reviews for Charlie's Restaurant & Bar (Boat Quay)
Reviews for Chef Kang's Private Kitchen
Reviews for Coucou Authentic Swiss Restaurant and Bar
Reviews for Crystal Jade Golden Palace
Reviews for Dark Horse Japanese Restaurant & Bar
Reviews for Empress
Reviews for Esquina
Reviews for FOC Restaurant
Reviews for Famous Treasure Chinese Restaurant
Reviews for Firangi Superstar
Reviews for Fleur De Sel Le Restaurant
Reviews for Forbidden Duck
Reviews for Forum Seafood Village Restaurant
Reviews for GEMMA Restaurant
Reviews for Gaig Singapore
Reviews for Godmama - Peranakan Restaurant Singapore
Reviews for Grand Shanghai Re